In [1]:
#DESCARGAMOS LOS MODULOS PARA PODER CORRER EL SCRIPT
import pandas as pd
import numpy as np
import requests
import config
import time
from datetime import datetime, date, timedelta
from scipy import stats
import matplotlib.pyplot as plt
import sys
import pickle
import functions as fn
from redash import get_info_redash
from analystcommunity import read_connection_data_warehouse as read_dw,connections_utils as cu,sheets
import datetime
from datetime import datetime,time,date,timedelta
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import os
import time
import glob
import math

Vamos a correr primero la data no estructurada de Posgrado 

In [2]:
#JUNTAMOS TODOS LOS ARCHIVOS EN UNO SOLO
folder='./Posgrado'
allFiles = glob.glob(folder + "/*.xlsx")
list_ = []
for file_ in allFiles:
    df = pd.read_excel(file_)
    list_.append(df)
frame = pd.concat(list_)
print(frame)

           ESTADÍSTICA DE EDUCACIÓN SUPERIOR - POSGRADO      Unnamed: 1  \
0                               CICLO ESCOLAR 2017-2018             NaN   
1     ESPECIALIDAD, MAESTRÍA Y DOCTORADO: LUGARES OF...             NaN   
2     ENTIDAD FEDERATIVA               \n           ...             NaN   
3                                        AGUASCALIENTES             NaN   
4                                                   NaN  AGUASCALIENTES   
...                                                 ...             ...   
1090                                                NaN             NaN   
1091                                                NaN             NaN   
1092                                                NaN             NaN   
1093                                      Total general             NaN   
1094  Fuente: ANUIES, elaboración propia con datos d...             NaN   

     Unnamed: 2                                Unnamed: 3  \
0           NaN                       

In [3]:
frame.head()

,ESTADÍSTICA DE EDUCACIÓN SUPERIOR - POSGRADO,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,CICLO ESCOLAR 2017-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"ESPECIALIDAD, MAESTRÍA Y DOCTORADO: LUGARES OF...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENTIDAD FEDERATIVA \n ...,NaN,NaN,NaN,NaN,Lugares Ofertados,Primer Ingreso Hombres,Primer Ingreso Mujeres,Primer Ingreso Total,Matrícula Hombres,Matrícula Mujeres,Matrícula Total,Modalidad,Institucion,Ciclo,Tipo
3,AGUASCALIENTES,NaN,NaN,NaN,NaN,722,254,320,574,469,679,1148,Escolarizada,Particular,2017_2018,Posgrado
4,NaN,AGUASCALIENTES,NaN,NaN,NaN,722,254,320,574,461,668,1129,Escolarizada,Particular,2017_2018,Posgrado


In [4]:
#NOMBRAMOS LAS COLUMANS PARA PODER EMEZAR A CONSTRUIR LA DB
frames = frame.rename(columns = {'ESTADÍSTICA DE EDUCACIÓN SUPERIOR - POSGRADO': 'ENTIDAD_FEDERATIVA', 'Unnamed: 1': 'MUNICIPIO',
                        'Unnamed: 2' : 'INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR', 'Unnamed: 3':'ESCUELA', 'Unnamed: 4' : 'PROGRAMA_DE_ESTUDIOS ','Unnamed: 5':'LUGARES_OFERTADOS',
                        'Unnamed: 6':'PRIMER_INGRESO_HOMBRES'  , 'Unnamed: 7':'PRIMER_INGRESO_MUJERES',
                        'Unnamed: 8':'PRIMER_INGRESO_TOTAL','Unnamed: 9':'MATRICULA_HOMBRES', 'Unnamed: 10':'MATRICULA_MUJERES',
                        'Unnamed: 11':'MATRICULA_TOTAL', 'Unnamed: 12':'MODALIDAD', 'Unnamed: 13':'INSTITUCION' ,      
                         'Unnamed: 14':'CICLO', 'Unnamed: 15':'TIPO'})

In [5]:
#ELIMINAMOS EL IDEX O,1,2 QUE SON TEXTO QUE NO NOS VA A SERVIR
frames_posgrado = frames.drop([0,1,2])

In [6]:
#ELIMINAMOS LA ULTIMA FILA DE CADA ARCHIVO
filtrar = frames_posgrado['ENTIDAD_FEDERATIVA']== 'Fuente: ANUIES, elaboración propia con datos de los Cuestionarios 911.9B. Ciclo escolar 2016-2017. Inicio de cursos.'
frames_posgrado = frames_posgrado.drop(index=frames_posgrado[filtrar].index)

In [7]:
#ELIMINAMOS LA ULTIMA FILA DE CADA ARCHIVO
filtrar = frames_posgrado['ENTIDAD_FEDERATIVA']== 'Fuente: ANUIES, elaboración propia con datos de los Cuestionarios 911.9B. Ciclo escolar 2017-2018. Inicio de cursos.'
frames_posgrado = frames_posgrado.drop(index=frames_posgrado[filtrar].index)

In [8]:
#ELIMINAMOS LA ULTIMA FILA DE CADA ARCHIVO
filtrar = frames_posgrado['ENTIDAD_FEDERATIVA']== 'Fuente: ANUIES, elaboración propia con datos de los Cuestionarios 911.9B. Ciclo escolar 2018-2019. Inicio de cursos.'
frames_posgrado = frames_posgrado.drop(index=frames_posgrado[filtrar].index)

In [9]:
#VERIFICAMOS QUE YA NO TENEMOS LA ULTIMA FILA DE TEXTO
frames_posgrado.tail()

,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR,ESCUELA,PROGRAMA_DE_ESTUDIOS,LUGARES_OFERTADOS,PRIMER_INGRESO_HOMBRES,PRIMER_INGRESO_MUJERES,PRIMER_INGRESO_TOTAL,MATRICULA_HOMBRES,MATRICULA_MUJERES,MATRICULA_TOTAL,MODALIDAD,INSTITUCION,CICLO,TIPO
1089,NaN,NaN,NaN,NaN,ESPECIALIDAD EN TECNOLOGÍAS INFORMÁTICAS EDUCA...,0,0,0,0,0,0,0,NoEscolarizada,Publico,2018_2019,Posgrado
1090,NaN,NaN,NaN,NaN,MAESTRÍA EN TECNOLOGÍA INFORMÁTICA EDUCATIVA,177,60,48,108,106,100,206,NoEscolarizada,Publico,2018_2019,Posgrado
1091,NaN,NaN,NaN,UNIDAD ACADÉMICA DE INGENIERÍA ELÉCTRICA,NaN,21,7,1,8,16,7,23,NoEscolarizada,Publico,2018_2019,Posgrado
1092,NaN,NaN,NaN,NaN,DOCTORADO EN INGENIERÍA Y TECNOLOGÍA APLICADA,21,7,1,8,16,7,23,NoEscolarizada,Publico,2018_2019,Posgrado
1093,Total general,NaN,NaN,NaN,NaN,11795,3298,4963,8261,7654,11965,19619,NoEscolarizada,Publico,2018_2019,Posgrado


In [11]:
#Llenamos los espacios vacios de las tablas con sus etiquetas correspondientes
frames_posgrado.ENTIDAD_FEDERATIVA.ffill(inplace=True)
frames_posgrado.MUNICIPIO.ffill(inplace=True)
frames_posgrado.INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR.ffill(inplace=True)
frames_posgrado.ESCUELA.ffill(inplace=True)

frames_posgrado
#frames.pivot(index="ENTIDAD_FEDERATIVA2",columns = "INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR", values = "PRIMER_INGRESO_TOTAL")


,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR,ESCUELA,PROGRAMA_DE_ESTUDIOS,LUGARES_OFERTADOS,PRIMER_INGRESO_HOMBRES,PRIMER_INGRESO_MUJERES,PRIMER_INGRESO_TOTAL,MATRICULA_HOMBRES,MATRICULA_MUJERES,MATRICULA_TOTAL,MODALIDAD,INSTITUCION,CICLO,TIPO
3,AGUASCALIENTES,NaN,NaN,NaN,NaN,722,254,320,574,469,679,1148,Escolarizada,Particular,2017_2018,Posgrado
4,AGUASCALIENTES,AGUASCALIENTES,NaN,NaN,NaN,722,254,320,574,461,668,1129,Escolarizada,Particular,2017_2018,Posgrado
5,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO GALILEA,NaN,NaN,15,3,2,5,3,2,5,Escolarizada,Particular,2017_2018,Posgrado
6,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO GALILEA,DIR. DE CIENCIAS HUMANÍSTICAS Y SOCIALES C. UN...,NaN,15,3,2,5,3,2,5,Escolarizada,Particular,2017_2018,Posgrado
7,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO GALILEA,DIR. DE CIENCIAS HUMANÍSTICAS Y SOCIALES C. UN...,MAESTRÍA EN DERECHO CIVIL,15,3,2,5,3,2,5,Escolarizada,Particular,2017_2018,Posgrado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE DOCENCIA SUPERIOR,ESPECIALIDAD EN TECNOLOGÍAS INFORMÁTICAS EDUCA...,0,0,0,0,0,0,0,NoEscolarizada,Publico,2018_2019,Posgrado
1090,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE DOCENCIA SUPERIOR,MAESTRÍA EN TECNOLOGÍA INFORMÁTICA EDUCATIVA,177,60,48,108,106,100,206,NoEscolarizada,Publico,2018_2019,Posgrado
1091,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE INGENIERÍA ELÉCTRICA,NaN,21,7,1,8,16,7,23,NoEscolarizada,Publico,2018_2019,Posgrado
1092,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE INGENIERÍA ELÉCTRICA,DOCTORADO EN INGENIERÍA Y TECNOLOGÍA APLICADA,21,7,1,8,16,7,23,NoEscolarizada,Publico,2018_2019,Posgrado


In [12]:
frames_posgrado.columns

Index(['ENTIDAD_FEDERATIVA', 'MUNICIPIO', 'INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR',
       'ESCUELA', 'PROGRAMA_DE_ESTUDIOS ', 'LUGARES_OFERTADOS',
       'PRIMER_INGRESO_HOMBRES', 'PRIMER_INGRESO_MUJERES',
       'PRIMER_INGRESO_TOTAL', 'MATRICULA_HOMBRES', 'MATRICULA_MUJERES',
       'MATRICULA_TOTAL', 'MODALIDAD', 'INSTITUCION', 'CICLO', 'TIPO'],
      dtype='object')

In [13]:
#Dropeamos los N/A, usamos programa de estudios como nuestro pivote
frames_posgrado.dropna(subset=['PROGRAMA_DE_ESTUDIOS '],inplace = True)

In [14]:
missing_values_count = frames_posgrado.isna().sum()
missing_values_count

ENTIDAD_FEDERATIVA                   0
MUNICIPIO                            0
INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR    0
ESCUELA                              0
PROGRAMA_DE_ESTUDIOS                 0
LUGARES_OFERTADOS                    0
PRIMER_INGRESO_HOMBRES               0
PRIMER_INGRESO_MUJERES               0
PRIMER_INGRESO_TOTAL                 0
MATRICULA_HOMBRES                    0
MATRICULA_MUJERES                    0
MATRICULA_TOTAL                      0
MODALIDAD                            0
INSTITUCION                          0
CICLO                                0
TIPO                                 0
dtype: int64

In [15]:
#PASAMOS LA BASE A UN EXCEL
frames_posgrado.to_excel(folder+'limpio.xlsx') #Write DateFrame back as Excel file

In [16]:
#JUNTAMOS TODOS LOS ARCHIVOS EN UNO SOLO
folder='./Licenciatura'
allFiles = glob.glob(folder + "/*.xlsx")
list_ = []
for file_ in allFiles:
    df = pd.read_excel(file_)
    list_.append(df)
frame = pd.concat(list_)
print(frame)

      ESTADÍSTICA DE EDUCACIÓN SUPERIOR - LICENCIATURA Y TÉCNICO SUPERIOR  \
0                                CICLO ESCOLAR 2017-2018                    
1      TÉCNICO SUPERIOR, LICENCIATURA EN EDUCACIÓN NO...                    
2      ENTIDAD FEDERATIVA               \n           ...                    
3                                         AGUASCALIENTES                    
4                                                    NaN                    
...                                                  ...                    
16864                                                NaN                    
16865                                                NaN                    
16866                                                NaN                    
16867                                      Total general                    
16868  Fuente: ANUIES, elaboración propia con datos d...                    

           Unnamed: 1 Unnamed: 2 Unnamed: 3                        Unnamed:

In [17]:
frame.head()

,ESTADÍSTICA DE EDUCACIÓN SUPERIOR - LICENCIATURA Y TÉCNICO SUPERIOR,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,CICLO ESCOLAR 2017-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"TÉCNICO SUPERIOR, LICENCIATURA EN EDUCACIÓN NO...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENTIDAD FEDERATIVA \n ...,NaN,NaN,NaN,NaN,Lugares Ofertados,Primer Ingreso Hombres,Primer Ingreso Mujeres,Primer Ingreso Total,Matrícula Hombres,Matrícula Mujeres,Matrícula Total,Modalidad,Institucion,Ciclo,Tipo
3,AGUASCALIENTES,NaN,NaN,NaN,NaN,18540,2027,2765,4792,5872,7342,13214,Escolarizada,Particular,2017_2018,Licenciatura
4,NaN,AGUASCALIENTES,NaN,NaN,NaN,18440,2011,2726,4737,5846,7269,13115,Escolarizada,Particular,2017_2018,Licenciatura


In [18]:
#NOMBRAMOS LAS COLUMANS PARA PODER EMEZAR A CONSTRUIR LA DB
frames = frame.rename(columns = {'ESTADÍSTICA DE EDUCACIÓN SUPERIOR - LICENCIATURA Y TÉCNICO SUPERIOR': 'ENTIDAD_FEDERATIVA','Unnamed: 1': 'MUNICIPIO',
                        'Unnamed: 2' : 'INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR', 'Unnamed: 3':'ESCUELA', 'Unnamed: 4' : 'PROGRAMA_DE_ESTUDIOS ','Unnamed: 5':'LUGARES_OFERTADOS',
                        'Unnamed: 6':'PRIMER_INGRESO_HOMBRES'  , 'Unnamed: 7':'PRIMER_INGRESO_MUJERES',
                        'Unnamed: 8':'PRIMER_INGRESO_TOTAL','Unnamed: 9':'MATRICULA_HOMBRES', 'Unnamed: 10':'MATRICULA_MUJERES',
                        'Unnamed: 11':'MATRICULA_TOTAL', 'Unnamed: 12':'MODALIDAD', 'Unnamed: 13':'INSTITUCION' ,      
                         'Unnamed: 14':'CICLO', 'Unnamed: 15':'TIPO'})

In [19]:
#ELIMINAMOS EL IDEX O,1,2 QUE SON TEXTO QUE NO NOS VA A SERVIR
frames = frames.drop([0,1,2])

In [20]:
#ELIMINAMOS LA ULTIMA FILA DE CADA ARCHIVO
filtrar = frames['ENTIDAD_FEDERATIVA']== 'Fuente: ANUIES, elaboración propia con datos de los Cuestionarios 911.9A. Ciclo escolar 2016-2017. Inicio de cursos.'
frames = frames.drop(index=frames[filtrar].index)

In [21]:
#ELIMINAMOS LA ULTIMA FILA DE CADA ARCHIVO
filtrar = frames['ENTIDAD_FEDERATIVA']== 'Fuente: ANUIES, elaboración propia con datos de los Cuestionarios 911.9A. Ciclo escolar 2017-2018. Inicio de cursos.'
frames = frames.drop(index=frames[filtrar].index)

In [22]:
#ELIMINAMOS LA ULTIMA FILA DE CADA ARCHIVO
filtrar = frames['ENTIDAD_FEDERATIVA']== 'Fuente: ANUIES, elaboración propia con datos de los Cuestionarios 911.9A. Ciclo escolar 2018-2019. Inicio de cursos.'
frames = frames.drop(index=frames[filtrar].index)

In [23]:
#VERIFICAMOS QUE YA NO TENEMOS LA ULTIMA FILA DE TEXTO
frames.tail()

,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR,ESCUELA,PROGRAMA_DE_ESTUDIOS,LUGARES_OFERTADOS,PRIMER_INGRESO_HOMBRES,PRIMER_INGRESO_MUJERES,PRIMER_INGRESO_TOTAL,MATRICULA_HOMBRES,MATRICULA_MUJERES,MATRICULA_TOTAL,MODALIDAD,INSTITUCION,CICLO,TIPO
16863,NaN,NaN,NaN,NaN,LICENCIATURA EN INGENIERÍA CIVIL,20,18,4,22,35,8,43,Escolarizada,Particular,2016_2017,Licenciatura
16864,NaN,NaN,NaN,NaN,LICENCIATURA EN MEDICINA GENERAL,150,67,79,146,188,241,429,Escolarizada,Particular,2016_2017,Licenciatura
16865,NaN,NaN,NaN,NaN,LICENCIATURA EN MERCADOTECNIA,15,7,5,12,20,13,33,Escolarizada,Particular,2016_2017,Licenciatura
16866,NaN,NaN,NaN,NaN,LICENCIATURA EN NUTRICIÓN,15,5,5,10,17,49,66,Escolarizada,Particular,2016_2017,Licenciatura
16867,Total general,NaN,NaN,NaN,NaN,160496393,132543,158821,291363,453315,534347,987661,Escolarizada,Particular,2016_2017,Licenciatura


In [24]:
#Llenamos los espacios vacios de las tablas con sus etiquetas correspondientes
frames.ENTIDAD_FEDERATIVA.ffill(inplace=True)
frames.MUNICIPIO.ffill(inplace=True)
frames.INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR.ffill(inplace=True)
frames.ESCUELA.ffill(inplace=True)

frames
#frames.pivot(index="ENTIDAD_FEDERATIVA2",columns = "INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR", values = "PRIMER_INGRESO_TOTAL")


,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR,ESCUELA,PROGRAMA_DE_ESTUDIOS,LUGARES_OFERTADOS,PRIMER_INGRESO_HOMBRES,PRIMER_INGRESO_MUJERES,PRIMER_INGRESO_TOTAL,MATRICULA_HOMBRES,MATRICULA_MUJERES,MATRICULA_TOTAL,MODALIDAD,INSTITUCION,CICLO,TIPO
3,AGUASCALIENTES,NaN,NaN,NaN,NaN,18540,2027,2765,4792,5872,7342,13214,Escolarizada,Particular,2017_2018,Licenciatura
4,AGUASCALIENTES,AGUASCALIENTES,NaN,NaN,NaN,18440,2011,2726,4737,5846,7269,13115,Escolarizada,Particular,2017_2018,Licenciatura
5,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,NaN,NaN,13500,270,505,775,474,901,1375,Escolarizada,Particular,2017_2018,Licenciatura
6,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,NaN,13500,270,505,775,474,901,1375,Escolarizada,Particular,2017_2018,Licenciatura
7,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ADMINISTRACIÓN DE EMPRESAS,1500,41,58,99,76,95,171,Escolarizada,Particular,2017_2018,Licenciatura
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16863,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE DURANGO A.C.,UNIVERSIDAD AUTÓNOMA DE DURANGO - CAMPUS ZACAT...,LICENCIATURA EN INGENIERÍA CIVIL,20,18,4,22,35,8,43,Escolarizada,Particular,2016_2017,Licenciatura
16864,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE DURANGO A.C.,UNIVERSIDAD AUTÓNOMA DE DURANGO - CAMPUS ZACAT...,LICENCIATURA EN MEDICINA GENERAL,150,67,79,146,188,241,429,Escolarizada,Particular,2016_2017,Licenciatura
16865,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE DURANGO A.C.,UNIVERSIDAD AUTÓNOMA DE DURANGO - CAMPUS ZACAT...,LICENCIATURA EN MERCADOTECNIA,15,7,5,12,20,13,33,Escolarizada,Particular,2016_2017,Licenciatura
16866,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE DURANGO A.C.,UNIVERSIDAD AUTÓNOMA DE DURANGO - CAMPUS ZACAT...,LICENCIATURA EN NUTRICIÓN,15,5,5,10,17,49,66,Escolarizada,Particular,2016_2017,Licenciatura


In [25]:
frames.columns

Index(['ENTIDAD_FEDERATIVA', 'MUNICIPIO', 'INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR',
       'ESCUELA', 'PROGRAMA_DE_ESTUDIOS ', 'LUGARES_OFERTADOS',
       'PRIMER_INGRESO_HOMBRES', 'PRIMER_INGRESO_MUJERES',
       'PRIMER_INGRESO_TOTAL', 'MATRICULA_HOMBRES', 'MATRICULA_MUJERES',
       'MATRICULA_TOTAL', 'MODALIDAD', 'INSTITUCION', 'CICLO', 'TIPO'],
      dtype='object')

In [26]:
#Dropeamos los N/A, usamos programa de estudios como nuestro pivote
frames.dropna(subset=['PROGRAMA_DE_ESTUDIOS '],inplace = True)
frames

,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR,ESCUELA,PROGRAMA_DE_ESTUDIOS,LUGARES_OFERTADOS,PRIMER_INGRESO_HOMBRES,PRIMER_INGRESO_MUJERES,PRIMER_INGRESO_TOTAL,MATRICULA_HOMBRES,MATRICULA_MUJERES,MATRICULA_TOTAL,MODALIDAD,INSTITUCION,CICLO,TIPO
7,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ADMINISTRACIÓN DE EMPRESAS,1500,41,58,99,76,95,171,Escolarizada,Particular,2017_2018,Licenciatura
8,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ARQUITECTURA,1500,43,14,57,77,28,105,Escolarizada,Particular,2017_2018,Licenciatura
9,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN COMERCIO INTERNACIONAL,0,0,0,0,1,5,6,Escolarizada,Particular,2017_2018,Licenciatura
10,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN CONTABILIDAD Y DERECHO FISCAL,0,0,0,0,4,7,11,Escolarizada,Particular,2017_2018,Licenciatura
11,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN CONTABILIDAD Y FINANZAS,1500,20,27,47,22,36,58,Escolarizada,Particular,2017_2018,Licenciatura
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16862,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE DURANGO A.C.,UNIVERSIDAD AUTÓNOMA DE DURANGO - CAMPUS ZACAT...,LICENCIATURA EN GASTRONOMÍA,17,9,8,17,16,23,39,Escolarizada,Particular,2016_2017,Licenciatura
16863,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE DURANGO A.C.,UNIVERSIDAD AUTÓNOMA DE DURANGO - CAMPUS ZACAT...,LICENCIATURA EN INGENIERÍA CIVIL,20,18,4,22,35,8,43,Escolarizada,Particular,2016_2017,Licenciatura
16864,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE DURANGO A.C.,UNIVERSIDAD AUTÓNOMA DE DURANGO - CAMPUS ZACAT...,LICENCIATURA EN MEDICINA GENERAL,150,67,79,146,188,241,429,Escolarizada,Particular,2016_2017,Licenciatura
16865,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE DURANGO A.C.,UNIVERSIDAD AUTÓNOMA DE DURANGO - CAMPUS ZACAT...,LICENCIATURA EN MERCADOTECNIA,15,7,5,12,20,13,33,Escolarizada,Particular,2016_2017,Licenciatura


In [27]:
missing_values_count = frames.isna().sum()
missing_values_count

ENTIDAD_FEDERATIVA                   0
MUNICIPIO                            0
INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR    0
ESCUELA                              0
PROGRAMA_DE_ESTUDIOS                 0
LUGARES_OFERTADOS                    0
PRIMER_INGRESO_HOMBRES               0
PRIMER_INGRESO_MUJERES               0
PRIMER_INGRESO_TOTAL                 0
MATRICULA_HOMBRES                    0
MATRICULA_MUJERES                    0
MATRICULA_TOTAL                      0
MODALIDAD                            0
INSTITUCION                          0
CICLO                                0
TIPO                                 0
dtype: int64

In [28]:
frames.columns

Index(['ENTIDAD_FEDERATIVA', 'MUNICIPIO', 'INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR',
       'ESCUELA', 'PROGRAMA_DE_ESTUDIOS ', 'LUGARES_OFERTADOS',
       'PRIMER_INGRESO_HOMBRES', 'PRIMER_INGRESO_MUJERES',
       'PRIMER_INGRESO_TOTAL', 'MATRICULA_HOMBRES', 'MATRICULA_MUJERES',
       'MATRICULA_TOTAL', 'MODALIDAD', 'INSTITUCION', 'CICLO', 'TIPO'],
      dtype='object')

In [29]:
frames_posgrado.columns

Index(['ENTIDAD_FEDERATIVA', 'MUNICIPIO', 'INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR',
       'ESCUELA', 'PROGRAMA_DE_ESTUDIOS ', 'LUGARES_OFERTADOS',
       'PRIMER_INGRESO_HOMBRES', 'PRIMER_INGRESO_MUJERES',
       'PRIMER_INGRESO_TOTAL', 'MATRICULA_HOMBRES', 'MATRICULA_MUJERES',
       'MATRICULA_TOTAL', 'MODALIDAD', 'INSTITUCION', 'CICLO', 'TIPO'],
      dtype='object')

In [72]:
#Una Vez que toda la data de estos ciclos escolares esta limpia, vamos a unir posgrado y licenciatura
frames_limpio= pd.concat([frames, frames_posgrado])

In [73]:
missing_values_count = frames_limpio.isna().sum()
missing_values_count

ENTIDAD_FEDERATIVA                   0
MUNICIPIO                            0
INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR    0
ESCUELA                              0
PROGRAMA_DE_ESTUDIOS                 0
LUGARES_OFERTADOS                    0
PRIMER_INGRESO_HOMBRES               0
PRIMER_INGRESO_MUJERES               0
PRIMER_INGRESO_TOTAL                 0
MATRICULA_HOMBRES                    0
MATRICULA_MUJERES                    0
MATRICULA_TOTAL                      0
MODALIDAD                            0
INSTITUCION                          0
CICLO                                0
TIPO                                 0
dtype: int64

In [74]:
frames_limpio.shape

(117027, 16)

In [75]:
frames_limpio.head()

,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR,ESCUELA,PROGRAMA_DE_ESTUDIOS,LUGARES_OFERTADOS,PRIMER_INGRESO_HOMBRES,PRIMER_INGRESO_MUJERES,PRIMER_INGRESO_TOTAL,MATRICULA_HOMBRES,MATRICULA_MUJERES,MATRICULA_TOTAL,MODALIDAD,INSTITUCION,CICLO,TIPO
7,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ADMINISTRACIÓN DE EMPRESAS,1500,41,58,99,76,95,171,Escolarizada,Particular,2017_2018,Licenciatura
8,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ARQUITECTURA,1500,43,14,57,77,28,105,Escolarizada,Particular,2017_2018,Licenciatura
9,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN COMERCIO INTERNACIONAL,0,0,0,0,1,5,6,Escolarizada,Particular,2017_2018,Licenciatura
10,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN CONTABILIDAD Y DERECHO FISCAL,0,0,0,0,4,7,11,Escolarizada,Particular,2017_2018,Licenciatura
11,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN CONTABILIDAD Y FINANZAS,1500,20,27,47,22,36,58,Escolarizada,Particular,2017_2018,Licenciatura


In [76]:
#ELIMINAMOS LA ULTIMA FILA DE CADA ARCHIVO
frames_limpio = frames_limpio.loc[~frames_limpio['PROGRAMA_DE_ESTUDIOS '].isin(['#¿NOMBRE?'])]
frames_limpio

,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR,ESCUELA,PROGRAMA_DE_ESTUDIOS,LUGARES_OFERTADOS,PRIMER_INGRESO_HOMBRES,PRIMER_INGRESO_MUJERES,PRIMER_INGRESO_TOTAL,MATRICULA_HOMBRES,MATRICULA_MUJERES,MATRICULA_TOTAL,MODALIDAD,INSTITUCION,CICLO,TIPO
7,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ADMINISTRACIÓN DE EMPRESAS,1500,41,58,99,76,95,171,Escolarizada,Particular,2017_2018,Licenciatura
8,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ARQUITECTURA,1500,43,14,57,77,28,105,Escolarizada,Particular,2017_2018,Licenciatura
9,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN COMERCIO INTERNACIONAL,0,0,0,0,1,5,6,Escolarizada,Particular,2017_2018,Licenciatura
10,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN CONTABILIDAD Y DERECHO FISCAL,0,0,0,0,4,7,11,Escolarizada,Particular,2017_2018,Licenciatura
11,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN CONTABILIDAD Y FINANZAS,1500,20,27,47,22,36,58,Escolarizada,Particular,2017_2018,Licenciatura
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE CONTADURÍA Y ADMINISTRACIÓN,ESPECIALIDAD EN VALUACIÓN CON ORIENTACIÓN EN N...,26,22,1,23,22,1,23,NoEscolarizada,Publico,2018_2019,Posgrado
1087,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE DERECHO,MAESTRÍA EN DOCENCIA E INVESTIGACIÓN JURÍDICAS,102,17,20,37,38,46,84,NoEscolarizada,Publico,2018_2019,Posgrado
1089,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE DOCENCIA SUPERIOR,ESPECIALIDAD EN TECNOLOGÍAS INFORMÁTICAS EDUCA...,0,0,0,0,0,0,0,NoEscolarizada,Publico,2018_2019,Posgrado
1090,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE DOCENCIA SUPERIOR,MAESTRÍA EN TECNOLOGÍA INFORMÁTICA EDUCATIVA,177,60,48,108,106,100,206,NoEscolarizada,Publico,2018_2019,Posgrado


In [77]:
frames_limpio.shape

(117026, 16)

In [78]:
#ELIMINAMOS LA ULTIMA FILA DE CADA ARCHIVO
frames_limpio = frames_limpio.loc[~frames_limpio['MATRICULA_TOTAL'].isin([0])]
frames_limpio

,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCIÓN_DE_EDUCACIÓN_SUPERIOR,ESCUELA,PROGRAMA_DE_ESTUDIOS,LUGARES_OFERTADOS,PRIMER_INGRESO_HOMBRES,PRIMER_INGRESO_MUJERES,PRIMER_INGRESO_TOTAL,MATRICULA_HOMBRES,MATRICULA_MUJERES,MATRICULA_TOTAL,MODALIDAD,INSTITUCION,CICLO,TIPO
7,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ADMINISTRACIÓN DE EMPRESAS,1500,41,58,99,76,95,171,Escolarizada,Particular,2017_2018,Licenciatura
8,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN ARQUITECTURA,1500,43,14,57,77,28,105,Escolarizada,Particular,2017_2018,Licenciatura
9,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN COMERCIO INTERNACIONAL,0,0,0,0,1,5,6,Escolarizada,Particular,2017_2018,Licenciatura
10,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN CONTABILIDAD Y DERECHO FISCAL,0,0,0,0,4,7,11,Escolarizada,Particular,2017_2018,Licenciatura
11,AGUASCALIENTES,AGUASCALIENTES,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,CENTRO UNIVERSITARIO BRITÁNICO DE MÉXICO,LICENCIATURA EN CONTABILIDAD Y FINANZAS,1500,20,27,47,22,36,58,Escolarizada,Particular,2017_2018,Licenciatura
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE CONTADURÍA Y ADMINISTRACIÓN,ESPECIALIDAD EN VALUACIÓN CON ORIENTACIÓN EN M...,16,14,2,16,16,2,18,NoEscolarizada,Publico,2018_2019,Posgrado
1085,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE CONTADURÍA Y ADMINISTRACIÓN,ESPECIALIDAD EN VALUACIÓN CON ORIENTACIÓN EN N...,26,22,1,23,22,1,23,NoEscolarizada,Publico,2018_2019,Posgrado
1087,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE DERECHO,MAESTRÍA EN DOCENCIA E INVESTIGACIÓN JURÍDICAS,102,17,20,37,38,46,84,NoEscolarizada,Publico,2018_2019,Posgrado
1090,ZACATECAS,ZACATECAS,UNIVERSIDAD AUTÓNOMA DE ZACATECAS,UNIDAD ACADÉMICA DE DOCENCIA SUPERIOR,MAESTRÍA EN TECNOLOGÍA INFORMÁTICA EDUCATIVA,177,60,48,108,106,100,206,NoEscolarizada,Publico,2018_2019,Posgrado


In [81]:
frames_limpio.shape

(106671, 16)

In [82]:
#PASAMOS TODA ESTA BASE LIMPIA A UN EXCEL
frames_limpio.to_excel('PosgradoyLic_Limpio_2016_2019.xlsx') #Write DateFrame back as Excel file